In [159]:
import requests
import re
from tqdm.auto import tqdm
import pandas as pd
import pymorphy2
import time

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

from sklearn.metrics.pairwise import linear_kernel

In [160]:
#скачиваем с api вакансии
sber = '3529'
page = 1
num_per_page = 100
ekaterinburg = 1261

url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={ekaterinburg}'
res = requests.get(url)
vacancies = res.json()
if vacancies is None:
        time.sleep(600)
        
# извлекаем Id вакансий
all_vacancy_ids = []
for i in tqdm(range(vacancies.get('pages'))):
    num_pages = vacancies.get('pages')
    vacancy_id = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_id)

  0%|          | 0/3 [00:00<?, ?it/s]

In [161]:
all_vacs= {}
all_vacids = all_vacancy_ids


for vacid in tqdm(all_vacids):
    url = f'https://api.hh.ru/vacancies/{vacid}?host=hh.ru'
    paramz = {
            'employer_id':sber,
            'per_page':num_per_page,
            'area': ekaterinburg,
        }
    res = requests.get(url, params=paramz)
    all_vacs[vacid]= res.json()

  0%|          | 0/300 [00:00<?, ?it/s]

In [162]:
tabl = {'id': [], 'name': [],'description': [], 'alternate_url': []}

In [163]:
for k, v in all_vacs.items():
    tabl['id'].append(k)
    tabl['name'].append(v.get('name'))
    tabl['description'].append(v.get('description'))
    tabl['alternate_url'].append(v.get('alternate_url'))
df = pd.DataFrame(tabl)

In [164]:
#предобрабатываем и добавляем преобразованный текст в новый столбец 'text_clean'
#удаляем пропуски
df.dropna(subset = ['description'], inplace = True)

In [165]:
# функция удаления пунктуации и цифр
def remove_trash(x): 
    pattern = '[^А-Яа-я0-9]+'
    try:
      x = [re.sub(pattern, ' ', i) for i in x] 
      print (x)
    except Exception as e:
      print(e)
    return x
df['text_clean'] = remove_trash(df['description'])

# функция удаления html-тэгов
def remove_trash(x): 
    pattern = r'<.*?>'
    try:
      x = [re.sub(pattern, ' ', i) for i in x] 
      print (x)
    except Exception as e:
      print(e)
    return x
df['text_clean'] = remove_trash(df['description'])

[' Клиентский менеджер специалист который продает продукты Сбера в отделении банка через планшет и терминалы самообслуживания Эта работа для тех у кого пока нет опыта зато есть амбиции и желание развиваться в продажах И мы в этом поможем У нас практически безграничные возможности горизонтального и вертикального роста даже в другом городе в любом регионе страны Ведь Сбер это более 50 компаний экосистемы и 14 тысяч подразделений по всей стране Регулярное корпоративное обучение Виртуальная школа с десятками курсов и современной библиотекой Корпоративный университет и возможность учиться за счет компании А ещ мы одни из лучших работодателей страны по версии Если хочешь построить карьеру и стать мастером эффективных продаж то работа клиентского менеджера в Сбере точно для тебя Что нужно будет делать продавать продукты банка и экосистемы физическим лицам помогать клиентам оформлять услуги на планшете личном устройстве клиента или на устройстве самообслуживания в офисе банка Ты подходишь нам 

In [166]:
# функция лемматизации - приводит слова в 1 форму
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:   #удаляет слова, которые меньше == 3 символа
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)
df['text_clean'] = df.apply(lemmatize,axis=1)

In [167]:
#удалить часто встречающиеся слова и стоп-слова
stop_russian = stopwords.words('russian')

#текст преобразуем в векторное представление
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=15000 )
text = text_transformer.fit_transform(df['text_clean'])

In [168]:
#что ищем
text_to_find = 'сбор данных из внутренних и внешних источников, подготовка данных к анализу, разработка моделей обработки и анализа данных, в том числе с применением алгоритмов машинного обучения в рамках аудиторских проверок.'

In [169]:
#создали матрицу по поиску
text_to_find_matrix = text_transformer.transform([text_to_find])

# расчет косинусного расстояния
cosine_similarities = linear_kernel(text, text_to_find_matrix).flatten()

# запишем все попарные результаты сравнений
df['cos_similarities'] = cosine_similarities

# и отсортируем по убыванию (т.к. cos(0)=1)
df = df.sort_values(by=['cos_similarities'], ascending=[0])

In [170]:
#выводим ТОП 10 подходящтх вакансий
df1 = df[['name', 'alternate_url', 'cos_similarities' ]].head(10)
df1

,name,alternate_url,cos_similarities
31,Аналитик данных (клиентская аналитика),https://hh.ru/vacancy/48480378,0.461386
15,Разработчик хранилищ данных,https://hh.ru/vacancy/49073609,0.163073
45,Системный аналитик (Wealth management),https://hh.ru/vacancy/49056195,0.104355
40,Системный аналитик (Private Banking),https://hh.ru/vacancy/48034991,0.103428
75,Инженер по автотестированию (АС ПАРУС),https://hh.ru/vacancy/48353887,0.090051
66,Senior Java Developer,https://hh.ru/vacancy/48363572,0.088755
72,Системный аналитик с функцией Delivery Lead,https://hh.ru/vacancy/48480549,0.083693
98,Системный аналитик middle,https://hh.ru/vacancy/48480383,0.070762
32,Инженер по сопровождению (дежурная смена),https://hh.ru/vacancy/49023886,0.069569
46,Главный инженер по сопровождению,https://hh.ru/vacancy/48011396,0.068010
